## **Red Neuronal YOLOv3 Personalizada**
Inciando con el entranamiendo de YOLOv3  y Google colab para detección de objetos especificos.

In [ ]:
#Verificamos que la GPU este habilitada y disponible
!nvidia-smi


## **Conexion a Google Drive**
Necesitamos vincular nuestro Google Colab con google Drive para realizar backups del entrenamiento.

In [ ]:
# Limpiamos y desmontamos google Drive por cualquier inicialización previa
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
drive.mount('/content/drive')
#El siguiente comando crea un Link simbolico que vamos a usar para acceder a nuestro drive como si escribieramos la ruta completa por ejemplo  /content/drive/My\ Drive/
!ln -s /content/drive/My\ Drive/ /MyGDrive
!ls /MyGDrive

## **Acceso a Dataset desde Google Drive**
Almacenamos nuestro dataset en Google Drive y lo transferimos a Google Colab.

In [ ]:
# Accedemos a nuestro drive para verificar que se tenga acceso a los archivos necesarios
!ls /content/drive/My\ Drive/INTELIGENCIA_ARTIFICIAL_AVANZADA/Yolov3


backup		   images      obj.data   yolov3_custom.cfg
generate_train.py  images.zip  obj.names  yolov3_ygb.cfg


In [ ]:
#Copiamos el archivo de images en nuestro colab (en la raiz de nuestra VM en la nube)
!cp /content/drive/My\ Drive/INTELIGENCIA_ARTIFICIAL_AVANZADA/Yolov3/images.zip /content/

## **1) Clonando la Darknet**

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

## **2) Compilando la Darknet usando Nvidia GPU**

In [ ]:
%cd darknet/

/content/darknet


In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

## **3) Configurando la red Darknet para entrenamiento YOLO V3**

In [ ]:
#Descargaremos el archivo de configuración (CFG) de Google Drive y le cambiaremos el nombre
%cd /content/darknet/

/content/darknet


In [ ]:
!cp /MyGDrive/INTELIGENCIA_ARTIFICIAL_AVANZADA/Yolov3/yolov3_ygb.cfg cfg/yolov3_ygb.cfg 
!cp /MyGDrive/INTELIGENCIA_ARTIFICIAL_AVANZADA/Yolov3/yolov3_custom.cfg cfg/yolov3_custom.cfg 

In [ ]:
#copiamos losr archivos de data y namos a la carpeta de darknet contienen la configuración necesaria
!cp /content/drive/My\ Drive/INTELIGENCIA_ARTIFICIAL_AVANZADA/Yolov3/obj.names /content/darknet/data/ 
!cp /content/drive/My\ Drive/INTELIGENCIA_ARTIFICIAL_AVANZADA/Yolov3/obj.data /content/darknet/data/ 

In [ ]:
#Descomprimimos nuestras imagenes para generar el archivo de entrenamiento
!unzip ../images.zip -d /content/darknet/data/colab/

unzip:  cannot find or open ../images.zip, ../images.zip.zip or ../images.zip.ZIP.


## **4) Generando el archivo train.txt**

In [ ]:
!cp /content/drive/My\ Drive/INTELIGENCIA_ARTIFICIAL_AVANZADA/Yolov3/generate_train.py ../


In [ ]:
#ejecutamos el script de python
%cd /content/
!python generate_train.py

/content
python3: can't open file 'generate_train.py': [Errno 2] No such file or directory


## 5) Descargamos pesos pre entrenados para capas de convolución


In [ ]:
# Descargar pesos de capas convolucionales preentrenados
!wget http://pjreddie.com/media/files/darknet53.conv.74

##6) Entrenamiento de nuetros detector de objetos!

In [ ]:
%cd /content/darknet
!./darknet detector train data/obj.data cfg/yolov3_ygb.cfg /content/darknet53.conv.74 -dont_show

In [ ]:
#Cada 100 paso se guarda una version de avance y posterior cada 1000 esto deberia guardase en neustra carpeta de backup
#imShow('chart.png')

Para que su modelo sea 'preciso', la pérdida debería estar por debajo de 2.

CONSEJO: El entrenamiento puede tomar varias horas dependiendo de cuántas iteraciones haya elegido en el archivo .cfg. Deberá dejar que esto se ejecute mientras duermes o trabajas durante el día, etc. Sin embargo, Colab Cloud Service lo inicia con sus máquinas virtuales si está inactivo durante demasiado tiempo (30-90 minutos).

Para evitar esta retención (CTRL + MAYÚS + i) al mismo tiempo, abra la vista del inspector en su navegador.

Pegue el siguiente código en la ventana de su consola y presione Enter
```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```
¡Es como si hicieras click en la pantalla cada 10 minutos para que no te desconecten por estar inactivo! HACKS!



Nota: Configura tu equipo para que no se suspenda o hiberne.

####  **6.1) Contiuar entranamiento desde archivo guardado**

TRUCO: Si por alguna razón genera un error o su Colab queda inactivo durante el entrenamiento, ¡no has perdido el archivo de weights! Cada 100 iteraciones se guarda un archivo de pesos llamado yolov3_custom_last.weights en la carpeta MyGDrive/yolov3/backup/ (o donde esté su carpeta de respaldo). Es por eso que creamos esta carpeta en nuestra unidad de Google y no en la nube VM. Si su tiempo de ejecución falla y su carpeta de respaldo estaba en su VM en la nube, si se perdería el archivo de weights y su progreso de entrenamiento.

¡Podemos comenzar el entrenamiento desde nuestro último archivo de pesos guardado para no tener que reiniciar! Simplemente ejecute el siguiente comando pero con su ubicación de respaldo.

In [ ]:
%cd /content/darknet
!./darknet detector train data/obj.data cfg/yolov3_ygb.cfg /MyGDrive/Yolov3/backup/yolov3_ygb_2000.weights -dont_show

## 7) Ejecutando nuestro Detector de Objetos!

In [ ]:
# Necesitamos configurar nuestro archivo custom.cfg para el modo de detección  
%cd /content/darknet/cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..

In [ ]:
# Ejecutamos nuestro detector perzonalizado con este comando (cargamos la(s) image(s) a nuestro google drive para hacer la prueba, la marca de umbral establece la precisión que debe tener la detección para mostrarla)
!./darknet detector test data/obj.data cfg/yolov3_custom.cfg /MyGDrive/Yolov3/backup/yolov3_ygb_last.weights /MyGDrive/Yolov3/img/d.jpg -thresh 0.3